In [1]:
import molli as ml
import molli.visual
import molli.visual.backend_pyvista
import pyvista
import numpy as np
import ipympl
import trame
import ipygany
import panel

In [2]:
selected = np.load("real_output/optimized/all_features.npy")
selected_IDW_1 = np.load("real_output/optimized/all_features_IDW_1.npy")
selected_IDW_2 = np.load("real_output/optimized/all_features_IDW_2.npy")
selected_IDW_3 = np.load("real_output/optimized/all_features_IDW_3.npy")

In [3]:
grid = np.load("grid.npy")

In [4]:
print(grid[0])

[ -8.54045 -14.14865 -10.2271 ]


In [5]:
print(selected[0])
print(selected_IDW_1[0])

[294   2]
[294.          3.2460377]


In [5]:
ids = selected[:, 0].astype(int)
freq = selected[:, 1].astype(np.float64)
filtered = grid[ids]
print(len(filtered))

314


In [6]:
pyvista.global_theme.trame.server_proxy_enabled = True
pyvista.global_theme.trame.server_proxy_prefix = "/proxy/"
pyvista.set_jupyter_backend('panel')

/tmp/ipykernel_572271/2154665095.py:3: PyVistaDeprecationWarning: `panel` backend is deprecated and is planned for future removal.
  pyvista.set_jupyter_backend('panel')


In [7]:
cloud = pyvista.PolyData(filtered)

In [9]:
print(cloud)

PolyData (0x7f355f2caf80)
  N Cells:    314
  N Points:   314
  N Strips:   0
  X Bounds:   -5.540e+00, 1.446e+01
  Y Bounds:   -1.415e+01, 9.851e+00
  Z Bounds:   -9.227e+00, 1.377e+01
  N Arrays:   0


In [8]:
cloud.point_data["values"] = freq

In [9]:
pyvista.start_xvfb()
plotter = pyvista.Plotter()

In [10]:
plotter.add_points(cloud, render_points_as_spheres=True, cmap='coolwarm', clim=[min(freq), max(freq)])
plotter.show()

BokehModel(combine_events=True, render_bundle={'docs_json': {'7cfc5ffa-0d1e-400b-b0ca-be2bed0906fe': {'version…

In [11]:
lib = ml.MoleculeLibrary("../nbo-cu-box-clean/cu-box-opt.mlib")

In [12]:
# NEW
lib = ml.ConformerLibrary("../nbo-cu-box-clean/caseys/ee_subset.mlib")

In [13]:
lib["aa_1"][0]

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

Conformer(name='aa_1', formula='C21 H18 N2 O2')

In [14]:
ml.visual.backend_pyvista.plot_structure_as_wireframe(plotter, lib["aa_1"][0])

In [15]:
plotter.show()

BokehModel(combine_events=True, render_bundle={'docs_json': {'fd1d9de2-d273-433e-b8d1-9a85a7f7f639': {'version…

In [17]:
plotter2 = pyvista.Plotter()
plotter2.add_points(cloud, render_points_as_spheres=True, cmap='coolwarm', clim=[min(freq), max(freq)])
plotter2.show()

BokehModel(combine_events=True, render_bundle={'docs_json': {'cb322433-7955-4bb9-aae0-85091b9a11f0': {'version…

In [18]:
ml.visual.backend_pyvista.plot_structure_as_wireframe(plotter2, lib["1_1_1_1"])
plotter2.show()

BokehModel(combine_events=True, render_bundle={'docs_json': {'bacfcb4d-0c09-44dc-96d5-a6dd10b83c97': {'version…

# Above are using wrong mlib

In [19]:
caseys = ml.ConformerLibrary("../nbo-cu-box-clean/caseys/conformers_no_linker.mlib")

In [20]:
print(caseys)
print(len(caseys))

96110


In [21]:
caseys["aa_1"][0]

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

Conformer(name='aa_1', formula='C21 H18 N2 O2')

In [22]:
better_plot = plotter2 = pyvista.Plotter()
better_plot.add_points(cloud, render_points_as_spheres=True, cmap='coolwarm', clim=[min(freq), max(freq)])
ml.visual.backend_pyvista.plot_structure_as_wireframe(better_plot, caseys["aa_1"][0])
better_plot.show()

BokehModel(combine_events=True, render_bundle={'docs_json': {'33d32283-1747-435d-9f05-6ac95fc415db': {'version…

Lack of symmetry in selected features likely due to correlated columns being dropped

# Now for clustering attempt

In [23]:
import pandas as pd

In [24]:
df = pd.read_json("real_output/optimized/_cluster/cluster_assignments.json", orient="split")

In [25]:
print(df)

       assignment
294             0
294             0
296             0
296             0
297             0
...           ...
15719           1
15720           1
15745           1
15955           1
16523           1

[393 rows x 1 columns]


In [26]:
df.reset_index(inplace=True)
df.drop_duplicates(["index"], inplace=True)
df.set_index(["index"], inplace=True)
print(df)

       assignment
index            
294             0
296             0
297             0
298             0
299             0
...           ...
15719           1
15720           1
15745           1
15955           1
16523           1

[314 rows x 1 columns]


In [27]:
cloud = pyvista.PolyData(filtered)
cloud.point_data["values"] = df.loc[:, "assignment"]

In [28]:
cluster_plot = pyvista.Plotter()
cluster_plot.add_points(cloud, render_points_as_spheres=True, cmap='coolwarm', clim=[0, 1])

Actor (0x7f34882c47c0)
  Center:                     (4.459549903869629, -2.1486501693725586, 2.272900104522705)
  Pickable:                   True
  Position:                   (0.0, 0.0, 0.0)
  Scale:                      (1.0, 1.0, 1.0)
  Visible:                    True
  X Bounds                    -5.540E+00, 1.446E+01
  Y Bounds                    -1.415E+01, 9.851E+00
  Z Bounds                    -9.227E+00, 1.377E+01
  User matrix:                Unset
  Has mapper:                 True

Property (0x7f34882c4880)
  Ambient:                     0.0
  Ambient color:               Color(name='white', hex='#ffffffff', opacity=255)
  Anisotropy:                  0.0
  Color:                       Color(name='white', hex='#ffffffff', opacity=255)
  Culling:                     "none"
  Diffuse:                     1.0
  Diffuse color:               Color(name='white', hex='#ffffffff', opacity=255)
  Edge color:                  Color(name='black', hex='#000000ff', opacity=255)
  In

In [29]:
ml.visual.backend_pyvista.plot_structure_as_wireframe(cluster_plot, caseys["aa_1"][0])
cluster_plot.show()

BokehModel(combine_events=True, render_bundle={'docs_json': {'de4cd4f4-068e-47c9-8951-2335c0c73950': {'version…

# Again but cleaned

In [30]:
cleaned = np.array([i for i in selected_IDW_2 if i[1].astype(np.float64) > 4.5])

In [31]:
ids_clean = cleaned[:, 0].astype(int)
freq_clean = cleaned[:, 1].astype(np.float64)
filtered_clean = grid[ids_clean]

In [32]:
cloud_clean = pyvista.PolyData(filtered_clean)
cloud_clean.point_data["values"] = df.loc[ids_clean.astype(str), "assignment"]
plotter_clean = pyvista.Plotter()

In [33]:
plotter_clean.add_points(cloud_clean, render_points_as_spheres=True, cmap='coolwarm', clim=[0, 1])
ml.visual.backend_pyvista.plot_structure_as_wireframe(plotter_clean, caseys["aa_1"][0])
plotter_clean.show()

BokehModel(combine_events=True, render_bundle={'docs_json': {'ef0041f5-0b1f-4b91-9acd-6449a28e58b5': {'version…